#Files type, Readind speed and size

In [ ]:
!pip install fastavro

In [5]:
import pandas as pd
import os
import time
import pickle
import fastavro
import pyarrow.orc as orc
import pyarrow.feather as feather


# Generating large dataset (1000000)
print("Generating large dataset...")
num_rows = 1000000
data = {
    'id': range(num_rows),
    'name': [f'User_{i}' for i in range(num_rows)],
    'value_a': [float(i * 1.5) for i in range(num_rows)],
    'category': [f'Category_{i % 10}' for i in range(num_rows)],
    'description': [f'This is a long description for item {i}' for i in range(num_rows)]
}
df = pd.DataFrame(data)
print(f"Dataset generated with {len(df)} rows and {len(df.columns)} columns.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.9 MB/s eta 0:00:00
Generating large dataset...
Dataset generated with 1000000 rows and 5 columns.


In [ ]:
print(df.head(5))

   id    name  value_a    category                            description
0   0  User_0      0.0  Category_0  This is a long description for item 0
1   1  User_1      1.5  Category_1  This is a long description for item 1
2   2  User_2      3.0  Category_2  This is a long description for item 2
3   3  User_3      4.5  Category_3  This is a long description for item 3
4   4  User_4      6.0  Category_4  This is a long description for item 4


## Testing Write Speed and File Size

In [6]:
# Make path for data
output_dir = 'data_speed_test'
os.makedirs(output_dir, exist_ok=True)

csv_file = os.path.join(output_dir, 'large_data.csv')
json_file = os.path.join(output_dir, 'large_data.json')
parquet_file = os.path.join(output_dir, 'large_data.parquet')
pickle_file = os.path.join(output_dir, 'large_data.pkl')
avro_file = os.path.join(output_dir, 'large_data.avro')
orc_file = os.path.join(output_dir, 'large_data.orc')
feather_file = os.path.join(output_dir, 'large_data.feather')

print("\n--- Testing Write Speed and File Size ---")

# Writing in CSV file
start_time = time.time()
df.to_csv(csv_file, index=False)
end_time = time.time()
print(f"CSV Write Time: {end_time - start_time:.4f} seconds")
print(f"CSV File Size: {os.path.getsize(csv_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in JSON file
start_time = time.time()
df.to_json(json_file, orient='records', lines=True)  # lines=True makes it JSONL (JSON Lines)
end_time = time.time()
print(f"JSON Write Time: {end_time - start_time:.4f} seconds")
print(f"JSON File Size: {os.path.getsize(json_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in Parquet file
start_time = time.time()
df.to_parquet(parquet_file, engine='pyarrow', index=False)
end_time = time.time()
print(f"Parquet Write Time: {end_time - start_time:.4f} seconds")
print(f"Parquet File Size: {os.path.getsize(parquet_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in ORC file
start_time = time.time()
df.to_orc(orc_file, engine='pyarrow')
end_time = time.time()
print(f"ORC Write Time: {end_time - start_time:.4f} seconds")
print(f"ORC File Size: {os.path.getsize(orc_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in Feather file
start_time = time.time()
df.to_feather(feather_file)
end_time = time.time()
print(f"Feather Write Time: {end_time - start_time:.4f} seconds")
print(f"Feather File Size: {os.path.getsize(feather_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in Pickle file
start_time = time.time()
with open(pickle_file, 'wb') as f:
    pickle.dump(df, f)
end_time = time.time()
print(f"Pickle Write Time: {end_time - start_time:.4f} seconds")
print(f"Pickle File Size: {os.path.getsize(pickle_file) / (1024 * 1024):.2f} MB")

print("- - -"*10)
# Writing in Avro file
schema = {
    "type": "record",
    "name": "DataFrame",
    "fields": [{"name": col, "type": "string" if df[col].dtype == 'object' else "int"} for col in df.columns]
}

start_time = time.time()
with open(avro_file, 'wb') as f:
    writer = fastavro.writer(f, schema, df.to_dict(orient="records"))
end_time = time.time()
print(f"Avro Write Time: {end_time - start_time:.4f} seconds")
print(f"Avro File Size: {os.path.getsize(avro_file) / (1024 * 1024):.2f} MB")

print("\n--- Testing Read Speed ---")


--- Testing Write Speed and File Size ---
CSV Write Time: 5.4887 seconds
CSV File Size: 78.13 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
JSON Write Time: 2.8544 seconds
JSON File Size: 130.58 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
Parquet Write Time: 0.7509 seconds
Parquet File Size: 18.24 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
ORC Write Time: 0.9696 seconds
ORC File Size: 67.56 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
Feather Write Time: 0.4326 seconds
Feather File Size: 28.41 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
Pickle Write Time: 1.7395 seconds
Pickle File Size: 83.73 MB
- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -
Avro Write Time: 9.3115 seconds
Avro File Size: 68.82 MB

--- Testing Read Speed ---


## Testing Reading Speed

In [7]:
# Reading from csv file
start_time = time.time()
df_csv = pd.read_csv(csv_file)
end_time = time.time()
print(f"CSV Read Time: {end_time - start_time:.4f} seconds")

# Reading from JSON file
start_time = time.time()
df_json = pd.read_json(json_file, orient='records', lines=True)
end_time = time.time()
print(f"JSON Read Time: {end_time - start_time:.4f} seconds")

# Reading from Parquet file (all columns)
start_time = time.time()
df_parquet = pd.read_parquet(parquet_file, engine='pyarrow')
end_time = time.time()
print(f"Parquet Read All Columns Time: {end_time - start_time:.4f} seconds")

# Reading from ORC file
start_time = time.time()
df_orc = pd.read_orc(orc_file)  # بدون تحديد محرك "engine"
end_time = time.time()
print(f"ORC Read Time: {end_time - start_time:.4f} seconds")

# Reading from Feather file
start_time = time.time()
df_feather = pd.read_feather(feather_file)
end_time = time.time()
print(f"Feather Read Time: {end_time - start_time:.4f} seconds")

# Reading from Pickle file
start_time = time.time()
with open(pickle_file, 'rb') as f:
    df_pickle = pickle.load(f)
end_time = time.time()
print(f"Pickle Read Time: {end_time - start_time:.4f} seconds")

# Reading from Avro file
start_time = time.time()
with open(avro_file, 'rb') as f:
    reader = fastavro.reader(f)
    df_avro = pd.DataFrame(list(reader))
end_time = time.time()
print(f"Avro Read Time: {end_time - start_time:.4f} seconds")

# Reading from Parquet file (only one row)
print("\n--- Testing Parquet Read Speed ---")
start_time = time.time()
df_parquet_col = pd.read_parquet(parquet_file, engine='pyarrow', columns=['value_a'])
end_time = time.time()
print(f"Parquet Read 'value_a' Column Only Time: {end_time - start_time:.4f} seconds")


CSV Read Time: 2.8576 seconds
JSON Read Time: 9.3556 seconds
Parquet Read All Columns Time: 1.1887 seconds
ORC Read Time: 0.9816 seconds
Feather Read Time: 0.8949 seconds
Pickle Read Time: 0.4047 seconds
Avro Read Time: 4.7201 seconds

--- Testing Parquet Read Speed ---
Parquet Read 'value_a' Column Only Time: 0.0539 seconds
